In [9]:
from __future__ import print_function
from IPython.display import display, HTML
import os
import sys
import csv
import time
import statistics
import numpy as np
import string
import re
import pandas as pd
import text_features_extractor as tfExtractor
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, accuracy_score, recall_score
#Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import plot_model
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import make_pipeline
import tensorflow
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import pickle

### NN model for Chinese data

In [10]:
#Read in the csv
chinDataDf = pd.read_csv("data_with_features/chin_cleaned_data_f.csv", encoding='UTF-8')
#Only take the text and sentiment columns
chinDataDf = chinDataDf[['text', 'depressed']]
#Cleaning
for index, row in chinDataDf.iterrows():
    #Preprocessing
    chinText, engText = tfExtractor.splitChinEng(row['text'])
    text = tfExtractor.chinPreprocessing(chinText)
    chinDataDf.set_value(index,'text',text)
#Convert data to numpy array
X = np.array(chinDataDf['text'].tolist())
Y = np.array(chinDataDf['depressed'].tolist())
#Convert -1 label to 0
i = 0
for label in Y:
    if(label == -1):
        Y[i] = 0
    i += 1

C:\Users\richi\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [11]:
#Original number of words: 46708
#Set top words
topWords = 5000
#Tokenizing the data
tokenizer = Tokenizer(num_words=topWords)
xString = []
for text in X:
    xString.append(' '.join(text))
tokenizer.fit_on_texts(xString)
print("tokenizer fitting is complete")
xSeq = tokenizer.texts_to_sequences(xString)
wordIndex = tokenizer.word_index
print("Number of words: " + str(len(wordIndex)))

tokenizer fitting is complete
Number of words: 31152


In [12]:
#Save tokenizer 
pickle.dump(tokenizer, open('saved_model/chinTokenizer.p', 'wb'))

In [13]:
#Get review mean length
lengths = [len(i) for i in xSeq]
print("review mean length: " + str(np.mean(lengths)))
#Set maximum review length to cover at least 90% of review content
maxReviewLength = int(np.percentile(lengths, 90))
print("maximum review length: " + str(maxReviewLength))

#Set paddings for review data
xPadded = pad_sequences(xSeq, maxlen=maxReviewLength)
print("Done padding")

review mean length: 273.42537313432837
maximum review length: 583
Done padding


In [14]:
def creatNnModel():
    # Simple multilayer perceptron model
    model = Sequential()
    #Embedding layer
    model.add(Embedding(topWords, 128, input_length=maxReviewLength))
    #Flattening
    model.add(Flatten())
    #Hidden layer
    model.add(Dense(250, activation='relu'))
    #Output layer
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #print(model.summary())
    return model

def createCnnModel():
    #CNN model
    cnnModel = Sequential()
    cnnModel.add(Embedding(topWords, 128, input_length=maxReviewLength))
    cnnModel.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
    cnnModel.add(MaxPooling1D(pool_size=2))
    cnnModel.add(Flatten())
    cnnModel.add(Dense(250, activation='relu'))
    cnnModel.add(Dense(1, activation='sigmoid'))
    cnnModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #print(cnnModel.summary())
    return cnnModel

In [15]:
nnAccuracy = []
nnPR = []
nnRecall = []
cnnAccuracy = []
cnnPR = []
cnnRecall = []
for i in range(5):
    print("Training session: " + str(i))
    #Split data into training and test set (80%/20%)
    xTrain, xTest, yTrain, yTest = train_test_split(xPadded, Y, test_size=0.2, shuffle=True, random_state=i, stratify=Y)
    
    #Create model
    model = creatNnModel()
    cnnModel = createCnnModel()
    
    # Fit the model
    model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=25, batch_size=50, verbose=2)
    # Final evaluation of the model
    scores = model.evaluate(xTest, yTest, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    yPred = model.predict(xTest)
    yPredNorm = []
    for p in yPred:
        if p > 0.5:
            yPredNorm.append(1)
        else:
            yPredNorm.append(0)
    average_precision = average_precision_score(yTest, yPred)
    nn_recall_score = recall_score(yTest, yPredNorm, average='binary')
    print('Average precision score: {0:0.2f}'.format(average_precision))
    print('Recall score: {0:0.2f}'.format(nn_recall_score))
    nnAccuracy.append(scores[1])
    nnPR.append(average_precision)
    nnRecall.append(nn_recall_score)
    
    # Fit the model
    cnnModel.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=25, batch_size=50, verbose=2)
    # Final evaluation of the model
    cnnscores = cnnModel.evaluate(xTest, yTest, verbose=0)
    yPred = cnnModel.predict(xTest)
    yPredNorm = []
    for p in yPred:
        if p > 0.5:
            yPredNorm.append(1)
        else:
            yPredNorm.append(0)
    cnn_average_precision = average_precision_score(yTest, yPred)
    cnn_recall_score = recall_score(yTest, yPredNorm, average='binary')
    print("Accuracy: %.2f%%" % (cnnscores[1]*100))
    print('Average precision score: {0:0.2f}'.format(cnn_average_precision))
    print('Recall score: {0:0.2f}'.format(cnn_recall_score))
    cnnAccuracy.append(cnnscores[1])
    cnnPR.append(cnn_average_precision)
    cnnRecall.append(cnn_recall_score)

Training session: 0
Train on 643 samples, validate on 161 samples
Epoch 1/25
 - 8s - loss: 0.8352 - acc: 0.5832 - val_loss: 0.6320 - val_acc: 0.6087
Epoch 2/25
 - 7s - loss: 0.5171 - acc: 0.7216 - val_loss: 0.5881 - val_acc: 0.6522
Epoch 3/25
 - 7s - loss: 0.3447 - acc: 0.8725 - val_loss: 0.5482 - val_acc: 0.7019
Epoch 4/25
 - 7s - loss: 0.2067 - acc: 0.9487 - val_loss: 0.5171 - val_acc: 0.7143
Epoch 5/25
 - 7s - loss: 0.1143 - acc: 0.9782 - val_loss: 0.4916 - val_acc: 0.7453
Epoch 6/25
 - 8s - loss: 0.0629 - acc: 0.9953 - val_loss: 0.4915 - val_acc: 0.7205
Epoch 7/25
 - 7s - loss: 0.0384 - acc: 0.9922 - val_loss: 0.5059 - val_acc: 0.7205
Epoch 8/25
 - 7s - loss: 0.0198 - acc: 0.9984 - val_loss: 0.5057 - val_acc: 0.7205
Epoch 9/25
 - 6s - loss: 0.0140 - acc: 0.9984 - val_loss: 0.4965 - val_acc: 0.7516
Epoch 10/25
 - 7s - loss: 0.0096 - acc: 0.9984 - val_loss: 0.5120 - val_acc: 0.7453
Epoch 11/25
 - 7s - loss: 0.0072 - acc: 1.0000 - val_loss: 0.5087 - val_acc: 0.7516
Epoch 12/25
 - 7s -

Epoch 19/25
 - 6s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.8851 - val_acc: 0.8447
Epoch 20/25
 - 6s - loss: 0.0013 - acc: 1.0000 - val_loss: 0.9056 - val_acc: 0.8447
Epoch 21/25
 - 6s - loss: 9.3060e-04 - acc: 1.0000 - val_loss: 0.9113 - val_acc: 0.8509
Epoch 22/25
 - 6s - loss: 7.5882e-04 - acc: 1.0000 - val_loss: 0.9274 - val_acc: 0.8447
Epoch 23/25
 - 6s - loss: 5.9329e-04 - acc: 1.0000 - val_loss: 0.9392 - val_acc: 0.8447
Epoch 24/25
 - 6s - loss: 5.3733e-04 - acc: 1.0000 - val_loss: 0.9513 - val_acc: 0.8447
Epoch 25/25
 - 6s - loss: 4.1992e-04 - acc: 1.0000 - val_loss: 0.9574 - val_acc: 0.8447
Accuracy: 84.47%
Average precision score: 0.82
Recall score: 0.76
Training session: 2
Train on 643 samples, validate on 161 samples
Epoch 1/25
 - 7s - loss: 1.0044 - acc: 0.5583 - val_loss: 0.6534 - val_acc: 0.6584
Epoch 2/25
 - 6s - loss: 0.5859 - acc: 0.7014 - val_loss: 0.6285 - val_acc: 0.6584
Epoch 3/25
 - 6s - loss: 0.5249 - acc: 0.6967 - val_loss: 0.6023 - val_acc: 0.6584
Epoch 4/25


Epoch 11/25
 - 6s - loss: 0.0095 - acc: 0.9953 - val_loss: 0.9081 - val_acc: 0.8137
Epoch 12/25
 - 6s - loss: 0.0039 - acc: 1.0000 - val_loss: 0.8914 - val_acc: 0.7888
Epoch 13/25
 - 6s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.9212 - val_acc: 0.8012
Epoch 14/25
 - 6s - loss: 0.0017 - acc: 1.0000 - val_loss: 0.9417 - val_acc: 0.8012
Epoch 15/25
 - 6s - loss: 0.0013 - acc: 1.0000 - val_loss: 0.9590 - val_acc: 0.8075
Epoch 16/25
 - 6s - loss: 0.0010 - acc: 1.0000 - val_loss: 0.9733 - val_acc: 0.8137
Epoch 17/25
 - 6s - loss: 7.5667e-04 - acc: 1.0000 - val_loss: 0.9908 - val_acc: 0.8137
Epoch 18/25
 - 6s - loss: 6.4081e-04 - acc: 1.0000 - val_loss: 1.0043 - val_acc: 0.8137
Epoch 19/25
 - 6s - loss: 5.1162e-04 - acc: 1.0000 - val_loss: 1.0151 - val_acc: 0.8137
Epoch 20/25
 - 6s - loss: 4.3179e-04 - acc: 1.0000 - val_loss: 1.0250 - val_acc: 0.8199
Epoch 21/25
 - 6s - loss: 3.7389e-04 - acc: 1.0000 - val_loss: 1.0332 - val_acc: 0.8199
Epoch 22/25
 - 6s - loss: 3.2709e-04 - acc: 1.0000 - val

In [16]:
nn_mean_ac = float(sum(nnAccuracy))/float(len(nnAccuracy))
nn_mean_pr = float(sum(nnPR))/float(len(nnPR))
nn_mean_recall = float(sum(nnRecall))/float(len(nnRecall))
cnn_mean_ac = float(sum(cnnAccuracy))/float(len(cnnAccuracy))
cnn_mean_pr = float(sum(cnnPR))/float(len(cnnPR))
cnn_mean_recall = float(sum(cnnRecall))/float(len(cnnRecall))
print("Overal NN result")
print("accuracy: " + str(nn_mean_ac))
print("precision score: " + str(nn_mean_pr))
print("recall: " + str(nn_mean_recall))
print("Overal CNN result")
print("accuracy: " + str(cnn_mean_ac))
print("precision score: " + str(cnn_mean_pr))
print("recall: " + str(cnn_mean_recall))

Overal NN result
accuracy: 0.7950310559006211
precision score: 0.7332347997823068
recall: 0.6181818181818182
Overal CNN result
accuracy: 0.8
precision score: 0.7820035808739274
recall: 0.6945454545454546


In [ ]:
plot_model(cnnModel, to_file='cnnModel.png')
plot_model(model, to_file='nnModel.png')

In [18]:
#Save the model
# serialize model to JSON
model_json = model.to_json()
with open("saved_model/chinModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("saved_model/chinModel.h5")
print("Saved model to disk")
cnnModel_json = cnnModel.to_json()
with open("saved_model/chinCnnModel.json", "w") as cnnJson_file:
    cnnJson_file.write(cnnModel_json)
# serialize weights to HDF5
cnnModel.save_weights("saved_model/chinCnnModel.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk


In [17]:
# load json and create model
json_file = open('saved_model/chinCnnModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_model/chinCnnModel.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(X, Y, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("embedding_4_1/random_uniform:0", shape=(5000, 128), dtype=float32)'

### NN model for English data

In [19]:
#Read in the csv
engDataDf = pd.read_csv("data_with_features/eng_cleaned_data_f.csv", encoding='UTF-8')
#Only take the text and sentiment columns
engDataDf = engDataDf[['text', 'depressed']]
#Cleaning
for index, row in engDataDf.iterrows():
    #Preprocessing
    chinText, engText = tfExtractor.splitChinEng(row['text'])
    text = tfExtractor.engPreprocessing(engText)
    engDataDf.set_value(index,'text',text)
#Convert data to numpy array
X = np.array(engDataDf['text'].tolist())
Y = np.array(engDataDf['depressed'].tolist())
#Convert -1 label to 0
i = 0
for label in Y:
    if(label == -1):
        Y[i] = 0
    i += 1

C:\Users\richi\Anaconda3\envs\python35\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [24]:
#Original number of words: 46708
#Set top words
topWords = 5000
#Tokenizing the data
tokenizer = Tokenizer(num_words=topWords)
tokenizer.fit_on_texts(X)
print("tokenizer fitting is complete")
xSeq = tokenizer.texts_to_sequences(X)
wordIndex = tokenizer.word_index
print("Number of words: " + str(len(wordIndex)))

tokenizer fitting is complete
Number of words: 7413


In [25]:
#Save tokenizer 
pickle.dump(tokenizer, open('saved_model/engTokenizer.p', 'wb'))

In [26]:
#Get review mean length
lengths = [len(i) for i in xSeq]
print("review mean length: " + str(np.mean(lengths)))
#Set maximum review length to cover at least 90% of review content
maxReviewLength = int(np.percentile(lengths, 90))
print("maximum review length: " + str(maxReviewLength))

#Set paddings for review data
xPadded = pad_sequences(xSeq, maxlen=maxReviewLength)
print("Done padding")

review mean length: 11.130532212885154
maximum review length: 17
Done padding


In [27]:
nnAccuracy = []
nnPR = []
nnRecall = []
cnnAccuracy = []
cnnPR = []
cnnRecall = []
for i in range(5):
    print("Training session: " + str(i))
    #Split data into training and test set (80%/20%)
    xTrain, xTest, yTrain, yTest = train_test_split(xPadded, Y, test_size=0.2, shuffle=True, random_state=i, stratify=Y)
    
    #Create model
    engModel = creatNnModel()
    engCnnModel = createCnnModel()
    
    # Fit the model
    engModel.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=20, batch_size=50, verbose=2)
    # Final evaluation of the model
    scores = engModel.evaluate(xTest, yTest, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    yPred = engModel.predict(xTest)
    yPredNorm = []
    for p in yPred:
        if p > 0.5:
            yPredNorm.append(1)
        else:
            yPredNorm.append(0)
    average_precision = average_precision_score(yTest, yPred)
    nn_recall_score = recall_score(yTest, yPredNorm, average='binary')
    print('Average precision-recall score: {0:0.2f}'.format(average_precision))
    nnAccuracy.append(scores[1])
    nnPR.append(average_precision)
    nnRecall.append(nn_recall_score)
    
    # Fit the model
    engCnnModel.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=20, batch_size=50, verbose=2)
    # Final evaluation of the model
    cnnscores = engCnnModel.evaluate(xTest, yTest, verbose=0)
    yPred = engCnnModel.predict(xTest)
    yPredNorm = []
    for p in yPred:
        if p > 0.5:
            yPredNorm.append(1)
        else:
            yPredNorm.append(0)
    cnn_average_precision = average_precision_score(yTest, yPred)
    cnn_recall_score = recall_score(yTest, yPredNorm, average='binary')
    print("Accuracy: %.2f%%" % (cnnscores[1]*100))
    print('Average precision score: {0:0.2f}'.format(cnn_average_precision))
    print('Recall score: {0:0.2f}'.format(cnn_recall_score))
    cnnAccuracy.append(cnnscores[1])
    cnnPR.append(cnn_average_precision)
    cnnRecall.append(cnn_recall_score)

Training session: 0
Train on 2856 samples, validate on 714 samples
Epoch 1/20
 - 2s - loss: 0.4846 - acc: 0.7546 - val_loss: 0.3246 - val_acc: 0.8585
Epoch 2/20
 - 1s - loss: 0.1590 - acc: 0.9468 - val_loss: 0.2850 - val_acc: 0.8487
Epoch 3/20
 - 2s - loss: 0.0349 - acc: 0.9937 - val_loss: 0.3008 - val_acc: 0.8754
Epoch 4/20
 - 1s - loss: 0.0131 - acc: 0.9972 - val_loss: 0.3295 - val_acc: 0.8557
Epoch 5/20
 - 2s - loss: 0.0088 - acc: 0.9986 - val_loss: 0.3498 - val_acc: 0.8641
Epoch 6/20
 - 2s - loss: 0.0057 - acc: 0.9993 - val_loss: 0.3764 - val_acc: 0.8571
Epoch 7/20
 - 1s - loss: 0.0059 - acc: 0.9986 - val_loss: 0.3785 - val_acc: 0.8627
Epoch 8/20
 - 1s - loss: 0.0032 - acc: 0.9993 - val_loss: 0.4150 - val_acc: 0.8501
Epoch 9/20
 - 1s - loss: 0.0042 - acc: 0.9993 - val_loss: 0.4224 - val_acc: 0.8515
Epoch 10/20
 - 2s - loss: 0.0044 - acc: 0.9993 - val_loss: 0.4179 - val_acc: 0.8557
Epoch 11/20
 - 1s - loss: 0.0050 - acc: 0.9986 - val_loss: 0.4401 - val_acc: 0.8557
Epoch 12/20
 - 1s 

Epoch 13/20
 - 2s - loss: 0.0063 - acc: 0.9986 - val_loss: 0.4269 - val_acc: 0.8627
Epoch 14/20
 - 2s - loss: 0.0084 - acc: 0.9982 - val_loss: 0.4450 - val_acc: 0.8627
Epoch 15/20
 - 2s - loss: 0.0043 - acc: 0.9986 - val_loss: 0.4182 - val_acc: 0.8585
Epoch 16/20
 - 2s - loss: 0.0042 - acc: 0.9989 - val_loss: 0.4378 - val_acc: 0.8599
Epoch 17/20
 - 2s - loss: 0.0057 - acc: 0.9989 - val_loss: 0.4239 - val_acc: 0.8585
Epoch 18/20
 - 2s - loss: 0.0050 - acc: 0.9982 - val_loss: 0.4455 - val_acc: 0.8515
Epoch 19/20
 - 1s - loss: 0.0046 - acc: 0.9989 - val_loss: 0.4563 - val_acc: 0.8613
Epoch 20/20
 - 2s - loss: 0.0052 - acc: 0.9986 - val_loss: 0.4421 - val_acc: 0.8557
Accuracy: 85.57%
Average precision-recall score: 0.95
Train on 2856 samples, validate on 714 samples
Epoch 1/20
 - 2s - loss: 0.4696 - acc: 0.7630 - val_loss: 0.3043 - val_acc: 0.8599
Epoch 2/20
 - 1s - loss: 0.1357 - acc: 0.9503 - val_loss: 0.2689 - val_acc: 0.8908
Epoch 3/20
 - 2s - loss: 0.0286 - acc: 0.9926 - val_loss: 0.2

Epoch 5/20
 - 2s - loss: 0.0054 - acc: 0.9982 - val_loss: 0.4456 - val_acc: 0.8655
Epoch 6/20
 - 2s - loss: 0.0070 - acc: 0.9982 - val_loss: 0.4654 - val_acc: 0.8585
Epoch 7/20
 - 1s - loss: 0.0072 - acc: 0.9982 - val_loss: 0.4742 - val_acc: 0.8557
Epoch 8/20
 - 2s - loss: 0.0061 - acc: 0.9986 - val_loss: 0.4758 - val_acc: 0.8627
Epoch 9/20
 - 2s - loss: 0.0036 - acc: 0.9989 - val_loss: 0.4987 - val_acc: 0.8557
Epoch 10/20
 - 2s - loss: 0.0130 - acc: 0.9975 - val_loss: 0.5184 - val_acc: 0.8571
Epoch 11/20
 - 2s - loss: 0.0068 - acc: 0.9989 - val_loss: 0.5396 - val_acc: 0.8599
Epoch 12/20
 - 2s - loss: 0.0047 - acc: 0.9989 - val_loss: 0.5716 - val_acc: 0.8613
Epoch 13/20
 - 2s - loss: 0.0039 - acc: 0.9989 - val_loss: 0.5735 - val_acc: 0.8627
Epoch 14/20
 - 2s - loss: 0.0032 - acc: 0.9989 - val_loss: 0.6055 - val_acc: 0.8627
Epoch 15/20
 - 2s - loss: 0.0039 - acc: 0.9989 - val_loss: 0.5760 - val_acc: 0.8529
Epoch 16/20
 - 2s - loss: 0.0032 - acc: 0.9989 - val_loss: 0.6429 - val_acc: 0.86

In [28]:
nn_mean_ac = float(sum(nnAccuracy))/float(len(nnAccuracy))
nn_mean_pr = float(sum(nnPR))/float(len(nnPR))
nn_mean_recall = float(sum(nnRecall))/float(len(nnRecall))
cnn_mean_ac = float(sum(cnnAccuracy))/float(len(cnnAccuracy))
cnn_mean_pr = float(sum(cnnPR))/float(len(cnnPR))
cnn_mean_recall = float(sum(cnnRecall))/float(len(cnnRecall))
print("Overal NN result")
print("accuracy: " + str(nn_mean_ac))
print("precision score: " + str(nn_mean_pr))
print("recall: " + str(nn_mean_recall))
print("Overal CNN result")
print("accuracy: " + str(cnn_mean_ac))
print("precision score: " + str(cnn_mean_pr))
print("recall: " + str(cnn_mean_recall))

Overal NN result
accuracy: 0.8414565825328773
precision score: 0.9400529799925967
recall: 0.8751322751322752
Overal CNN result
accuracy: 0.8649859943309751
precision score: 0.9449751197709093
recall: 0.8851851851851851


In [29]:
#Save the model
# serialize model to JSON
model_json = engModel.to_json()
with open("saved_model/engModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
engModel.save_weights("saved_model/engModel.h5")
print("Saved model to disk")
cnnModel_json = engCnnModel.to_json()
with open("saved_model/engCnnModel.json", "w") as cnnJson_file:
    cnnJson_file.write(cnnModel_json)
# serialize weights to HDF5
engCnnModel.save_weights("saved_model/engCnnModel.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk


In [ ]:
#Test Lime
class_names = ['not_depressed', 'depressed']
explainer = LimeTextExplainer(class_names=class_names)
texts = ["I am depressed, I am sick, hate every moment of my life", "I love today, best day of my life, super happy"]

In [ ]:
yPred = cnnModel.predict(xTest)
average_precision = average_precision_score(yTest, yPred)

print('Average precision-recall score: {0:0.2f}'.format(average_precision))